In [158]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.pipeline import Pipeline,make_pipeline

In [159]:
df=pd.read_csv('data\cleaned_1.csv')
df=df.drop('Unnamed: 0',axis=1)
df

,region,province,zip_code,property_type,property_subtype,price,number_rooms,living_area,kitchen,furnished,fireplace,terrace,terrace_area,garden,garden_area,surface_land,number_facades,swimming_pool,building_state
0,Flanders,Flemish Brabant,1970,HOUSE,HOUSE,460000.0,3.0,120.000000,USA_HYPER_EQUIPPED,0,0,1,20.000000,1,110.0,227.000000,2.0,0,GOOD
1,Brussels,Brussels,1150,HOUSE,HOUSE,599000.0,4.0,203.000000,INSTALLED,0,0,1,0.000000,1,50.0,258.000000,3.0,0,GOOD
2,Flanders,Antwerp,2970,HOUSE,HOUSE,655000.0,2.0,235.000000,INSTALLED,0,0,0,0.000000,0,0.0,930.000000,4.0,0,AS_NEW
3,Brussels,Brussels,1000,HOUSE,MIXED_USE_BUILDING,1475000.0,6.0,518.000000,NOT_INSTALLED,0,0,1,74.041614,1,200.0,330.000000,2.0,0,TO_RENOVATE
4,Wallonie,Liège,4500,HOUSE,HOUSE,149500.0,3.0,100.000000,SEMI_EQUIPPED,0,0,0,0.000000,0,0.0,265.000000,3.0,0,GOOD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16467,Flanders,East Flanders,9600,HOUSE,HOUSE,167950.0,2.0,209.515373,NOT_INSTALLED,0,0,0,0.000000,0,0.0,80.000000,2.0,0,GOOD
16468,Flanders,West Flanders,8500,HOUSE,HOUSE,162000.0,2.0,209.515373,NOT_INSTALLED,0,0,1,0.000000,0,0.0,139.000000,2.0,0,0
16469,Wallonie,Namur,5530,HOUSE,HOUSE,269000.0,3.0,106.000000,SEMI_EQUIPPED,0,0,0,0.000000,1,454.0,529.000000,4.0,0,0
16470,Flanders,Limburg,3850,HOUSE,HOUSE,629000.0,5.0,313.000000,HYPER_EQUIPPED,0,0,0,0.000000,0,0.0,8520.784087,4.0,0,0


In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16472 entries, 0 to 16471
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   region            16472 non-null  object 
 1   province          16472 non-null  object 
 2   zip_code          16472 non-null  object 
 3   property_type     16472 non-null  object 
 4   property_subtype  16472 non-null  object 
 5   price             16472 non-null  float64
 6   number_rooms      16472 non-null  float64
 7   living_area       16472 non-null  float64
 8   kitchen           16472 non-null  object 
 9   furnished         16472 non-null  int64  
 10  fireplace         16472 non-null  int64  
 11  terrace           16472 non-null  int64  
 12  terrace_area      16472 non-null  float64
 13  garden            16472 non-null  int64  
 14  garden_area       16472 non-null  float64
 15  surface_land      16472 non-null  float64
 16  number_facades    16472 non-null  float6

Location is very important for the price prodiction, so check abnormal values to remove

In [161]:
df['region'].unique()

array(['Flanders', 'Brussels', 'Wallonie', '0'], dtype=object)

In [162]:
#drop the rows of region that with 0 value
df=df[df['region']!='0']
df['region'].value_counts()

region
Flanders    9032
Wallonie    5529
Brussels    1800
Name: count, dtype: int64

In [163]:
df['province'].unique()

array(['Flemish Brabant', 'Brussels', 'Antwerp', 'Liège', 'Hainaut',
       'West Flanders', 'East Flanders', 'Namur', 'Luxembourg',
       'Walloon Brabant', 'Limburg'], dtype=object)

In [164]:
# Converting the postal code to integer 
def convert_zipcode(zip_code):
    """function to convert the postal code to integer and handel value error"""
    try:
        return int(zip_code)
    except ValueError:
        return None

# applying the above function to the column "zip_code "
df["zip_code"] = df["zip_code"].apply(convert_zipcode)
df["zip_code"]=df["zip_code"]/100
df["zip_code"]=df["zip_code"].round(0).astype(int)
df["zip_code"].unique()


C:\Users\frede_0021xgx\AppData\Local\Temp\ipykernel_12044\1314125693.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["zip_code"] = df["zip_code"].apply(convert_zipcode)
C:\Users\frede_0021xgx\AppData\Local\Temp\ipykernel_12044\1314125693.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["zip_code"]=df["zip_code"]/100
C:\Users\frede_0021xgx\AppData\Local\Temp\ipykernel_12044\1314125693.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

array([ 20,  12,  30,  10,  45,  71,  88,  90,  87,  54,  68,  93,  13,
        14,  56,  18,  19,  64,  62,  92,  38,  76,  67,  80,  70,  98,
        84,  15,  86,  85,  34,  16,  82,  91,  49,  11,  41,  33,  48,
        22,  29,  32,  69,  40,  61,  17,  44,  79,  53,  83,  60,  94,
        95,  25,  47,  23,  31,  26,  50,  21,  89, 100,  36,  73,  28,
        99,  51,  63,  57,  37,  46,  43,  97,  78,  39,  42,  35,  24,
        52,  66,  55,  96,  75,  72,  74,  65,  77,  27])

Check another abnormal values

In [165]:
df['property_type'].unique()

array(['HOUSE', 'APARTMENT'], dtype=object)

In [166]:
df['property_subtype'].unique()

array(['HOUSE', 'MIXED_USE_BUILDING', 'VILLA', 'EXCEPTIONAL_PROPERTY',
       'CHALET', 'APARTMENT', 'TRIPLEX', 'APARTMENT_BLOCK',
       'COUNTRY_COTTAGE', 'MANSION', 'FLAT_STUDIO', 'DUPLEX', 'CASTLE',
       'BUNGALOW', 'GROUND_FLOOR', 'TOWN_HOUSE', 'PENTHOUSE',
       'OTHER_PROPERTY', 'SERVICE_FLAT', 'MANOR_HOUSE', 'LOFT',
       'FARMHOUSE', 'KOT'], dtype=object)

In [167]:
df['living_area'].unique()

array([120.        , 203.        , 235.        , 518.        ,
       100.        , 154.        , 140.        , 127.        ,
       206.        , 180.        , 130.        , 260.        ,
       185.        , 148.        , 207.        , 135.        ,
       167.        , 181.        , 300.        , 233.        ,
       172.        , 445.        , 230.        , 240.        ,
        60.        , 125.        , 126.        , 209.51537287,
       143.        , 200.        , 209.        , 150.        ,
        96.        , 264.        , 270.        ,  54.        ,
        42.        ,  81.        , 166.        , 113.        ,
        98.        , 177.        , 124.        ,  65.        ,
       105.        , 115.        , 110.        , 215.        ,
       286.        , 400.        , 315.        , 350.        ,
       341.        , 225.        , 346.        , 305.        ,
       210.        , 324.        , 599.72512009, 275.        ,
       280.        , 363.        , 258.        , 369.  

reduce the number of types of kitchen and build state 

In [168]:
#checking types of kitchen
df['kitchen'].unique()

array(['USA_HYPER_EQUIPPED', 'INSTALLED', 'NOT_INSTALLED',
       'SEMI_EQUIPPED', 'HYPER_EQUIPPED', 'USA_INSTALLED',
       'USA_SEMI_EQUIPPED', 'USA_UNINSTALLED'], dtype=object)

In [169]:
#reduce to 3 values
df['kitchen']=df['kitchen'].replace({'USA_HYPER_EQUIPPED':'EQUIPED','SEMI_EQUIPPED':'EQUIPED',
                          'HYPER_EQUIPPED':'EQUIPED','HYPER_EQUIPPED':'EQUIPED','USA_SEMI_EQUIPPED':'EQUIPED',
                          'USA_INSTALLED':'INSTALLED','USA_UNINSTALLED':'NOT_INSTALLED'})

df['kitchen'].unique()

C:\Users\frede_0021xgx\AppData\Local\Temp\ipykernel_12044\1618473523.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kitchen']=df['kitchen'].replace({'USA_HYPER_EQUIPPED':'EQUIPED','SEMI_EQUIPPED':'EQUIPED',


array(['EQUIPED', 'INSTALLED', 'NOT_INSTALLED'], dtype=object)

In [170]:
#checking types of building_state
df['building_state'].unique()
#reduce to 3 values
df['building_state']=df['building_state'].replace({'TO_RESTORE':'TO_RENOVATE','JUST_RENOVATED':'AS_NEW',
                          'TO_BE_DONE_UP':'TO_RENOVATE','0':'UNKNOWN'})

df['building_state'].unique()

C:\Users\frede_0021xgx\AppData\Local\Temp\ipykernel_12044\1341711992.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['building_state']=df['building_state'].replace({'TO_RESTORE':'TO_RENOVATE','JUST_RENOVATED':'AS_NEW',


array(['GOOD', 'AS_NEW', 'TO_RENOVATE', 'UNKNOWN'], dtype=object)

In [171]:
#checking number_rooms
df['number_rooms'].unique()
#round up the float
df['number_rooms'] = df['number_rooms'].apply(np.ceil)
df['number_rooms'].unique()

C:\Users\frede_0021xgx\AppData\Local\Temp\ipykernel_12044\1929883001.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['number_rooms'] = df['number_rooms'].apply(np.ceil)


array([3., 4., 2., 6., 5., 1., 0., 7., 8.])

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16361 entries, 0 to 16471
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   region            16361 non-null  object 
 1   province          16361 non-null  object 
 2   zip_code          16361 non-null  int32  
 3   property_type     16361 non-null  object 
 4   property_subtype  16361 non-null  object 
 5   price             16361 non-null  float64
 6   number_rooms      16361 non-null  float64
 7   living_area       16361 non-null  float64
 8   kitchen           16361 non-null  object 
 9   furnished         16361 non-null  int64  
 10  fireplace         16361 non-null  int64  
 11  terrace           16361 non-null  int64  
 12  terrace_area      16361 non-null  float64
 13  garden            16361 non-null  int64  
 14  garden_area       16361 non-null  float64
 15  surface_land      16361 non-null  float64
 16  number_facades    16361 non-null  float64
 17

In [222]:
categorical_features=df.select_dtypes(include=object)
numeric_features=df.select_dtypes(include=float)
skip_features=df.select_dtypes(include=int)
list(categorical_features.columns)

['region',
 'province',
 'property_type',
 'property_subtype',
 'kitchen',
 'building_state']

In [215]:
# dividing the columns to prediction value and features
X = df.drop("price",axis=1)
y = df.price
# Shapes of X and y
print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)

#Splitting the train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
# Shapes of diffrent train and test sets
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)



Shape of X:  (16361, 18)
Shape of y:  (16361,)
Shape of X_train:  (13088, 18)
Shape of y_train:  (13088,)
Shape of X_test:  (3273, 18)
Shape of y_test:  (3273,)


In [223]:
# first transformation for the pipeline which is OneHotEncoder
trans_1 = ColumnTransformer([('encode',OneHotEncoder(dtype='int',sparse_output=False, handle_unknown='ignore'), 
                              list(categorical_features.columns))], remainder='passthrough' )


In [217]:
# second transformation for the pipeline which is scaling
trans_2 = ColumnTransformer([('scale', MinMaxScaler(), slice(0,18))], remainder='passthrough')

In [218]:
# third transformation for the pipeline which is creating an instance from linear models
trans_3 = LinearRegression()

In [224]:
# creating a pipeline with above transformer
pipe = Pipeline([('trans_1',trans_1),
                 ('trans_2',trans_2),
                 ('trans_3',trans_3),
                ])

In [225]:
# Training the model
pipe.fit(X_train, y_train)

Pipeline(steps=[('trans_1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encode',
                                                  OneHotEncoder(dtype='int',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['region', 'province',
                                                   'property_type',
                                                   'property_subtype',
                                                   'kitchen',
                                                   'building_state'])])),
                ('trans_2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 18, None))])),
                ('trans_3', LinearRegression())])

In [226]:
# printing the score
pipe.score(X_train, y_train)

0.4884075232321744

In [229]:
# predicting the test dataset
y_pred = pipe.predict(X_test)

In [228]:
pipe.score(X_test, y_test)

0.474193723819325